In [1]:
import gc
import os
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pickle
from sklearn.utils import resample
from collections import Counter
import math
from sklearn.model_selection import train_test_split
import helper
import dataset
import torch
import torch.nn as nn
from resnet1d.net1d import Net1D

In [2]:
data_files = [name for name in os.listdir('./data')]

dfs = []

for i in range(len(data_files)):
    fname = './data/' + data_files[i]
    df = pd.read_csv(fname)
    # df.columns = [x.strip().lstrip() for x in df.columns]

    dfs.append(df)

df = pd.concat(dfs, axis=0, ignore_index=True)

df.shape

(2830743, 79)

In [3]:
# Remove spaces in the front and the end of the column names for better human reading
df.columns = [x.lstrip().strip().replace('�', '-') for x in df.columns]
df.shape

(2830743, 79)

In [4]:
# replace inf values to nan
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df.shape

(2827876, 79)

In [5]:
# remove this when experimenting on the whole dataset
df = resample(df, replace=False, n_samples=20000, stratify=df['Label'])

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

pca__n_components = 20

preprocessor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("normalize", MinMaxScaler()),
    ("PCA", PCA(n_components=pca__n_components))
])

label_encoder = LabelEncoder()

In [7]:
X = df.drop(['Label'], axis=1)
y = df['Label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)

# doing this since we need all labels in case there's any labels not found in test set
label_encoder.fit(y)

y_train = label_encoder.transform(y_train)
y_val = label_encoder.transform(y_val)

In [8]:
X_smote, _ = helper.smote_sampling(X_train, y_train)
X_gaussion = helper.add_gaussion(X_smote)
X_flip = helper.invert_array(X_train)

X_new = np.dstack([X_smote, X_gaussion, X_flip])
X_new = X_new.transpose((0, 2, 1))

X_new.shape

(16000, 3, 20)

In [9]:
# batch, channel, feature
X_train = np.expand_dims(X_train, axis=2).transpose((0, 2, 1))
X_train.shape

(16000, 1, 20)

In [10]:
org_dataset = dataset.NumpyDataset(X_train, y_train)
pos_dataset = dataset.NumpyDataset(X_new, y_train)

In [14]:
from torch.utils.data import DataLoader
from pytorch_metric_learning.losses import SelfSupervisedLoss, NTXentLoss
from models import Extractor, Projector

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

extractor_1d = Extractor(n_features=pca__n_components, n_channels=1).to(device)
extractor_3d = Extractor(n_features=pca__n_components, n_channels=3).to(device)

projector = Projector().to(device)

lr = 1e-3
weight_decay = 1e-4
batch_size = 128
epochs = 200

org_dataloader = DataLoader(org_dataset, batch_size=batch_size,)
pos_dataloader = DataLoader(pos_dataset, batch_size=batch_size)

optimizer_1d = torch.optim.Adam(extractor_1d.parameters(), lr=lr, weight_decay=weight_decay)
optimizer_3d = torch.optim.Adam(extractor_3d.parameters(), lr=lr, weight_decay=weight_decay)
optimizer_projector = torch.optim.Adam(projector.parameters(), lr=lr, weight_decay=weight_decay)

loss_fn = NTXentLoss()
loss_fn = SelfSupervisedLoss(loss_fn, symmetric=False).to(device)

In [15]:
from itertools import cycle
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir='./runs')

cnt = 1

for epoch in range(epochs):
    total_loss = 0
    for item1, item2 in zip(org_dataloader, cycle(pos_dataloader)):
        X_batch, _ = item1
        X_new_batch, _ = item2

        X_batch = X_batch.to(device)
        X_new_batch = X_new_batch.to(device)

        optimizer_1d.zero_grad()
        optimizer_3d.zero_grad()
        optimizer_projector.zero_grad()

        embedding_1d = extractor_1d(X_batch)
        embedding_3d = extractor_3d(X_new_batch)
        projected_1d = projector(embedding_1d)
        projected_3d = projector(embedding_3d)

        loss = loss_fn(projected_1d, projected_3d)
        loss.backward()

        total_loss += loss.item()
    
        optimizer_1d.step()
        optimizer_3d.step()
        optimizer_projector.step()

    cnt += 1

    print(f'[{epoch}/{epochs}]: {total_loss}')
    
    writer.add_scalar('Loss/train', total_loss, cnt)

[0/200]: 526.4330174922943
[1/200]: 360.79523730278015
[2/200]: 326.10967469215393
[3/200]: 308.4385435581207
[4/200]: 292.6241066455841
[5/200]: 274.16411900520325
[6/200]: 257.2951933145523
[7/200]: 244.62671315670013
[8/200]: 229.48354721069336
[9/200]: 215.93647027015686
[10/200]: 206.07626259326935
[11/200]: 197.43839848041534
[12/200]: 187.03276336193085
[13/200]: 186.04748022556305
[14/200]: 174.28682219982147
[15/200]: 175.02989542484283
[16/200]: 171.25295209884644
[17/200]: 167.69488072395325
[18/200]: 161.35639584064484
[19/200]: 160.70035004615784
[20/200]: 161.0859353542328
[21/200]: 154.04450166225433
[22/200]: 148.15520495176315
[23/200]: 148.78121149539948
[24/200]: 149.76501202583313
[25/200]: 148.18415957689285
[26/200]: 141.98402070999146
[27/200]: 139.79235023260117
[28/200]: 139.27884125709534
[29/200]: 136.90944987535477
[30/200]: 135.4210729598999
[31/200]: 134.62163192033768
[32/200]: 130.68104380369186
[33/200]: 128.15140742063522
[34/200]: 133.04172468185425
[

In [16]:
X_val.shape

(4000, 20)

torch.Size([4000, 20])

In [19]:
X_val_tensor = torch.tensor(np.expand_dims(X_val, axis=2).transpose((0, 2, 1)), dtype=torch.float32, device=device)

X_val_embedding = extractor_1d(X_val_tensor)

In [23]:
X_val_embedding = X_val_embedding.cpu().detach().numpy()
X_val_embedding.shape

(4000, 64)

In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate

mlp = MLPClassifier(max_iter=500)

cross_validate(mlp, X_val_embedding, y_val, cv=5, scoring='f1_macro')

# mlp.fit(X_val_embedding, y_val)



/home/chan/miniconda3/envs/csi5388/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'fit_time': array([2.96512651, 3.01782084, 3.64208198, 3.65459371, 4.23379874]),
 'score_time': array([0.00241208, 0.00238132, 0.00236225, 0.00237226, 0.00239182]),
 'test_score': array([0.67431162, 0.81376944, 0.59952126, 0.59288618, 0.46378186])}

In [27]:
X_val.shape

(4000, 20)

In [26]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate

mlp = MLPClassifier(max_iter=500)

cross_validate(mlp, X_val, y_val, cv=5, scoring='f1_macro')

# mlp.fit(X_val_embedding, y_val)



{'fit_time': array([8.59820437, 3.76568174, 2.80605435, 2.94945192, 3.74656677]),
 'score_time': array([0.03523374, 0.03223753, 0.03021288, 0.03036594, 0.03034711]),
 'test_score': array([0.56215391, 0.55339958, 0.48419142, 0.60216681, 0.41635716])}